## Training neural networks

This notebook describes the workflow for training neural networks

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import getcwd
from pathlib import Path
from functools import reduce

import yaml
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from dstools.reproducibility import make_filename
from dstools.reproducibility.util import git_hash_in_path, get_version
from dstools.util import save

from yass import util

Using TensorFlow backend.


In [3]:
here = getcwd()
here_version = git_hash_in_path(here)

In [4]:
# for reference
print('YASS version is: {}'.format(get_version('yass')))
print('nbs version is: {}'.format(here_version))

YASS version is: 9220361 fixes make spikes function
nbs version is: a786da4 Merge branch 'master' of https://github.com/edublancas/yass-private


In [7]:
path_to_data = Path('~', 'data', 'detect').expanduser()
path_to_output = path_to_data  / 'models'
path_to_set = path_to_data / 'training-set.npz'

## Loading training data

In [9]:
from sklearn.model_selection import train_test_split

data = np.load(path_to_set)

X = data['X']
y = data['y']

x_train, x_test, y_train, y_test  = train_test_split(X, y, test_size=0.33)

In [10]:
def make_model(x_train, input_shape):
    n_data, window_size, n_channels, _ = x_train.shape

    model = Sequential()
        
#     model.add(MaxPooling2D(pool_size=(2, 1), data_format="channels_last", padding='same'))
    
#     model.add(Dropout(0.75))

    model.add(Conv2D(10, kernel_size=(5, 5),
                     padding='same', activation='relu', use_bias=True,
                     data_format="channels_last", input_shape=input_shape))


#     model.add(Conv2D(70, kernel_size=(window_size, 1),
#                      padding='valid', activation='relu', use_bias=True,
#                      data_format="channels_last"))
    
    model.add(Conv2D(10, kernel_size=(5, 5),
                     padding='same', activation='relu', use_bias=True,
                     data_format="channels_last"))


#     model.add(Conv2D(70, kernel_size=(1, n_channels),
#                      padding='valid', activation='relu', use_bias=True,
#                      data_format="channels_last"))
    
    model.add(Conv2D(10, kernel_size=(5, 5),
                     padding='same', activation='linear', use_bias=True,
                     data_format="channels_last"))
        
#     model.add(MaxPooling2D(pool_size=(1, 2), data_format="channels_last", padding='same'))
#     model.add(Dropout(0.75))
    
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    model.summary()
    
    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    opt = keras.optimizers.adam(lr=0.001)

    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    return model


# Training

In [11]:
# import models
x_train = x_train[:, : , :, np.newaxis]
x_test = x_test[:, : , :, np.newaxis]

_, wf, ch, _ = x_train.shape

m = make_model(x_train, (wf, ch, 1))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 61, 7, 10)         260       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 61, 7, 10)         2510      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 61, 7, 10)         2510      
_________________________________________________________________
flatten_1 (Flatten)          (None, 4270)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 4271      
Total params: 9,551
Trainable params: 9,551
Non-trainable params: 0
_________________________________________________________________


In [14]:
m.fit(x_train, y_train,
      batch_size=1000, epochs=100, shuffle=True,
      validation_data=(x_test, y_test))

Train on 36984 samples, validate on 18216 samples
Epoch 1/100
36984/36984 [==============================] - 3s 86us/step - loss: 0.1179 - acc: 0.9626 - val_loss: 0.0177 - val_acc: 0.9954
Epoch 2/100
36984/36984 [==============================] - 2s 58us/step - loss: 0.0137 - acc: 0.9965 - val_loss: 0.0141 - val_acc: 0.9959
Epoch 3/100
36984/36984 [==============================] - 2s 58us/step - loss: 0.0100 - acc: 0.9975 - val_loss: 0.0104 - val_acc: 0.9971
Epoch 4/100
36984/36984 [==============================] - 2s 59us/step - loss: 0.0084 - acc: 0.9979 - val_loss: 0.0100 - val_acc: 0.9973
Epoch 5/100
36984/36984 [==============================] - 2s 59us/step - loss: 0.0069 - acc: 0.9982 - val_loss: 0.0068 - val_acc: 0.9979
Epoch 6/100
36984/36984 [==============================] - 2s 59us/step - loss: 0.0057 - acc: 0.9985 - val_loss: 0.0064 - val_acc: 0.9980
Epoch 7/100
36984/36984 [==============================] - 2s 59us/step - loss: 0.0052 - acc: 0.9985 - val_loss: 0.0065 - 

KeyboardInterrupt: 

In [16]:
metadata = dict(path_to_set=path_to_set,
                yass_version=get_version('yass'),
                nb_version=git_hash_in_path(getcwd()))
metadata

{'path_to_set': PosixPath('/home/Edu/data/detect/training-set.npz'),
 'yass_version': '9220361 fixes make spikes function',
 'nb_version': "a786da4 Merge branch 'master' of https://github.com/edublancas/yass-private"}

In [19]:
_, wf, ch, _ = m.input_shape

sufix = f'detect-{wf}wf{ch}ch'
names = make_filename(sufix=sufix, extension=('h5', 'yaml'))

path_to_model, path_to_metadata = [str(path_to_output / name) for name in names]

m.save(path_to_model)
save(metadata, path_to_metadata)

In [20]:
print(path_to_model)

/home/Edu/data/detect/models/2018-10-03T04-13-50:detect-61wf7ch.h5
